<a href="https://colab.research.google.com/github/LilyZWuZz/DataScience-And-MachineLearning-Handbook-For-Coders/blob/master/tianchi_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving fomatted.csv to fomatted.csv
Saving jinnan_round1_testA_20181227.csv to jinnan_round1_testA_20181227.csv
Saving jinnan_round1_train_20181227.csv to jinnan_round1_train_20181227.csv
User uploaded file "fomatted.csv" with length 394316 bytes
User uploaded file "jinnan_round1_testA_20181227.csv" with length 36306 bytes
User uploaded file "jinnan_round1_train_20181227.csv" with length 344852 bytes


In [0]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# import plotly.graph_objs as go
# import plotly.tools as tls
import seaborn as sbn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

In [4]:
train = pd.read_csv('jinnan_round1_train_20181227.csv', encoding = 'gb18030')
test  = pd.read_csv('jinnan_round1_testA_20181227.csv', encoding = 'gb18030')
print(train.shape)
print(test.shape)
print(train.head(5))

(1396, 44)
(150, 43)
          样本id   A1  A2     A3   A4        A5    A6   A7  A8        A9  A10  \
0  sample_1528  300 NaN  405.0  700  13:30:00  38.0  NaN NaN  15:30:00  100   
1  sample_1698  300 NaN  405.0  700  14:00:00  29.0  NaN NaN  16:00:00  101   
2   sample_639  300 NaN  405.0  700  14:00:00  29.0  NaN NaN  16:00:00  102   
3   sample_483  300 NaN  405.0  700   1:30:00  38.0  NaN NaN   3:00:00  100   
4   sample_617  300 NaN  405.0  700  22:00:00  29.0  NaN NaN   0:00:00  101   

        A11  A12  A13       A14    A15       A16    A17  A18  A19  \
0  16:30:00  102  0.2  17:30:00  103.0  18:30:00  104.0  0.2  300   
1  17:00:00  103  0.2  18:00:00  104.0  19:00:00  105.0  0.2  200   
2  17:00:00  103  0.2  18:00:00  104.0  19:00:00  105.0  0.2  200   
3   4:00:00  102  0.2   5:00:00  103.0   6:00:00  104.0  0.2  200   
4   1:00:00  103  0.2   2:00:00  104.0   3:00:00  105.0  0.2  200   

           A20   A21   A22  A23       A24 A25       A26  A27          A28  \
0  21:00-21:

In [0]:
#处理缺失数据
# 删除类别唯一数据
for df in [train, test]:
    df.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)

    

In [0]:
#删除类别基本不变的值
good_cols = list(train.columns)
# for col in train.columns:
#     rate = train[col].value_counts(normalize=True, dropna=False).values[0]
#     if rate > 0.9:
#         good_cols.remove(col)
#         print(col,rate)

# 删除异常值
train = train[train['收率']>0.87].reset_index(drop=True)

In [7]:
train_Y = train["收率"]
good_cols.remove('收率')
train_X = train[good_cols]
test_X = test[good_cols]

print(test_X.shape,train_X.shape,train_Y.shape)
print(train_X.head())
print(test_X.head())

(150, 38) (1381, 38) (1381,)
          样本id   A1  A2     A3   A4        A5    A6   A7  A8        A9  A10  \
0  sample_1528  300 NaN  405.0  700  13:30:00  38.0  NaN NaN  15:30:00  100   
1  sample_1698  300 NaN  405.0  700  14:00:00  29.0  NaN NaN  16:00:00  101   
2   sample_639  300 NaN  405.0  700  14:00:00  29.0  NaN NaN  16:00:00  102   
3   sample_483  300 NaN  405.0  700   1:30:00  38.0  NaN NaN   3:00:00  100   
4   sample_617  300 NaN  405.0  700  22:00:00  29.0  NaN NaN   0:00:00  101   

        A11  A12       A14    A15       A16    A17  A19          A20   A21  \
0  16:30:00  102  17:30:00  103.0  18:30:00  104.0  300  21:00-21:30  50.0   
1  17:00:00  103  18:00:00  104.0  19:00:00  105.0  200  19:00-20:00  50.0   
2  17:00:00  103  18:00:00  104.0  19:00:00  105.0  200  19:00-19:30  50.0   
3   4:00:00  102   5:00:00  103.0   6:00:00  104.0  200    6:30-7:00  50.0   
4   1:00:00  103   2:00:00  104.0   3:00:00  105.0  200    3:00-4:00  50.0   

    A22       A24 A25      

In [8]:
data = pd.read_csv('fomatted.csv')
X_train = data.iloc[:train_X.shape[0],:]
X_test  = data.iloc[train_X.shape[0]:,:]
print(data.shape,X_train.shape,X_test.shape)
X_train = X_train.values
y_train = train_Y.values
X_test = X_test.values

(1531, 52) (1381, 52) (150, 52)


In [0]:
folds = KFold(n_splits=7, shuffle=True, random_state=2018)

In [10]:
##### xgb 'eta': 0.005, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8, 
xgb_params ={'eta': 0.005, 'max_depth': 10, 'n_estimators':5000,
              'subsample': 0.8, 'min_child_weight':1,
              'colsample_bytree': 0.8, 'colsample_bylevel': 0.9,
              'reg_alpha': 0, 'reg_lambda':0.65,
              'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': -1}


oof_xgb = np.zeros(len(X_train))
#predictions_xgb = np.zeros(len(X_test))
best_xgb = np.zeros((folds.n_splits,len(X_test)))
i=0
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=4000, evals=watchlist, early_stopping_rounds=300, verbose_eval=200, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    #predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    best_xgb[i]=clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit)
    i+=1
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, y_train)))

fold n°1
[0]	train-rmse:0.423163	valid_data-rmse:0.422776
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 300 rounds.
[200]	train-rmse:0.156582	valid_data-rmse:0.155811
[400]	train-rmse:0.059183	valid_data-rmse:0.058989
[600]	train-rmse:0.023846	valid_data-rmse:0.024655
[800]	train-rmse:0.011218	valid_data-rmse:0.013853
[1000]	train-rmse:0.006851	valid_data-rmse:0.011166
[1200]	train-rmse:0.005244	valid_data-rmse:0.010605
[1400]	train-rmse:0.004359	valid_data-rmse:0.010494
[1600]	train-rmse:0.00365	valid_data-rmse:0.01045
[1800]	train-rmse:0.00307	valid_data-rmse:0.010419
[2000]	train-rmse:0.002605	valid_data-rmse:0.010408
[2200]	train-rmse:0.002195	valid_data-rmse:0.010417
Stopping. Best iteration:
[2014]	train-rmse:0.002571	valid_data-rmse:0.010405

fold n°2
[0]	train-rmse:0.423333	valid_data-rmse:0.421772
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early sto

In [0]:
#best is the fold6
predictions_xgb = best_xgb[5]

In [12]:
param = {'learning_rate': 0.01,
         'max_depth': -1,
         'n_estimators': 3000,
         'num_leaves': 120,
         'min_child_samples': 30,
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "boosting": "gbdt",
         "bagging_seed": 2019,
         'objective':'regression',
         "metric": 'rmse', 
         "verbosity": -1}

oof_lgb = np.zeros(len(X_train))
#predictions_lgb = np.zeros(len(test))
best_lgb = np.zeros((folds.n_splits,len(test)))
i=0
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 5000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=300, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    #predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
    best_lgb[i]=clf.predict(X_test, num_iteration=clf.best_iteration)
    i+=1

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, y_train)))

fold n°1
Training until validation scores don't improve for 200 rounds.
[300]	training's rmse: 0.0105385	valid_1's rmse: 0.0115215
[600]	training's rmse: 0.00826866	valid_1's rmse: 0.0104527
[900]	training's rmse: 0.00710031	valid_1's rmse: 0.0101887
[1200]	training's rmse: 0.0062508	valid_1's rmse: 0.0101128
[1500]	training's rmse: 0.00558411	valid_1's rmse: 0.0101047
[1800]	training's rmse: 0.00504346	valid_1's rmse: 0.0100972
Early stopping, best iteration is:
[1707]	training's rmse: 0.00519462	valid_1's rmse: 0.0100833
fold n°2
Training until validation scores don't improve for 200 rounds.
[300]	training's rmse: 0.0103353	valid_1's rmse: 0.0132693
[600]	training's rmse: 0.00802778	valid_1's rmse: 0.0122314
[900]	training's rmse: 0.00680089	valid_1's rmse: 0.012125
Early stopping, best iteration is:
[901]	training's rmse: 0.00679711	valid_1's rmse: 0.0121243
fold n°3
Training until validation scores don't improve for 200 rounds.
[300]	training's rmse: 0.0103995	valid_1's rmse: 0.011

In [0]:
# best is fold1
predictions_lgb = best_lgb[0]

In [14]:
# 将lgb和xgb的结果进行stacking
train_stack = np.vstack([oof_lgb,oof_xgb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb]).transpose()

folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,train_Y)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], train_Y.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], train_Y.iloc[val_idx].values
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
mean_squared_error(y_train, oof_stack)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9


0.00012222010120275427

In [0]:
count=0
def save_test(test,pred):
    res=pd.DataFrame({'样本id':test['样本id'], '产率':pred})
    res['产率'] = res['产率'].apply(lambda x:round(x, 3))
    name = 'test_result_%d.csv' %count
    res.to_csv(name,header=False,index=False)
save_test(test.iloc[:test.shape[0],:],predictions)

# 新段落

In [0]:
from google.colab import files

files.download('test_result_0.csv')